In [8]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
import time

count = 0

def 連接到資料庫(db):
    global sql,conn,cursor,ids
    # 連接到資料庫A ====================================================================
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    sql = '''select * from financialStatements1 t'''

    # 連接到資料庫B
    sqlStock = '''select * from stock t'''
    dfStock = pd.read_sql(sqlStock,conn)
    dfStock = pd.DataFrame(dfStock)
    ids = dfStock['code'][count:]
    # ===================================================================================
    
def 爬蟲(id):
    dataExport = pd.DataFrame()
    dataExport = pd.DataFrame(columns=['代碼','季度','營業毛利率','營業利益率','稅前淨利率','稅後淨利率','稅後淨利率(母公司)',
    '每股稅前盈餘(元)','每股稅後盈餘(元)','每股淨值(元)','股東權益報酬率(季累計)','股東權益報酬率(年預估)','資產報酬率(季累計)',
    '資產報酬率(年預估)','營收年成長率','毛利年成長率','營業利益年成長率','稅前淨利年成長率','稅後淨利年成長率','稅後淨利年成長率(母公司)',
    '每股稅後盈餘年成長率','現金(%)','應收帳款(%)','存貨(%)','速動資產(%)','流動資產(%)','基金與投資(%)','固定資產(%)',
    '無形資產(%)','其他資產(%)','現金季成長率','應收帳款季成長率','存貨季成長率','流動資產季成長率','基金與投資季成長率',
    '固定資產季成長率','無形資產季成長率','其他資產季成長率','資產總額季成長率','現金年成長率','應收帳款年成長率',
    '存貨年成長率','流動資產年成長率','基金與投資年成長率','固定資產年成長率','無形資產年成長率','其他資產年成長率',
    '資產總額年成長率','應付帳款(%)','流動負債(%)','長期負債(%)','其他負債(%)','負債總額(%)','普通股股本(%)','股東權益總額(%)',
    '應付帳款季成長率','流動負債季成長率','長期負債季成長率','其他負債季成長率','負債總額季成長率','普通股股本季成長率',
    '股東權益總額季成長率','應付帳款年成長率','流動負債年成長率','長期負債年成長率','其他負債年成長率','負債總額年成長率',
    '普通股股本年成長率','股東權益總額年成長率','現金比','速動比','流動比','利息保障倍數','現金流量比(季累計)',
    '現金流量比(年預估)','營業成本率','營業費用率','應收帳款週轉率(次/年)','應收款項收現日數(日)','應付帳款週轉率(次/年)',
    '應付款項付現日數(日)','存貨週轉率(次/年)','平均售貨日數(日)','固定資產週轉率(次/年)','總資產週轉率(次/年)',
    '淨值週轉率(次/年)','應收帳款佔營收比率(季累計)','應收帳款佔營收比率(年預估)','存貨佔營收比率(季累計)',
    '存貨佔營收比率(年預估)','每股營業現金流量(元)','每股投資現金流量(元)','每股融資現金流量(元)','每股淨現金流量(元)',
    '每股自由現金流量(元)','負債對淨值比率','長期資金適合率','所得稅佔稅前淨利比率','業外損益佔營收比率',
    '業外損益佔稅前淨利比率','財報評分(100為滿分)'])
    
    url = f'https://goodinfo.tw/tw/StockFinDetail.asp?RPT_CAT=XX_M_QUAR_ACC&STOCK_ID={id}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.27'}
    res = requests.get(url, headers = headers,timeout=15)
    res.encoding = "utf-8" 
    
    soup = BeautifulSoup(res.text,"lxml") 
    data = soup.select_one("#txtFinBody")
        
    # 【重整表格】 ====================================================================
    dfs = pd.read_html(data.prettify())
    df = dfs[1]
    df2 = df

    df2_columns = df2.columns
    df2_columns = ['季度' if x == '獲利能力' else x for x in df2_columns]

    # 将列名添加为新的一行
    df2.loc[-1] = df2_columns

    # 将索引值加一，将新行移动到第一行
    df2.index = df2.index + 1
    df2 = df2.sort_index()
    df2 = df2.reset_index(drop=True)

    # 修改columns名稱，先計算總共有幾列
    z = ['index']
    for x in range(len(df2.columns)-1):
        z.append(x)
    df2.columns = z

    # Transpose DataFrame
    df3 = df2.copy()
    df3.set_index('index',inplace=True)
    df3 = df3.T
    df3.insert(0,'代碼',id)

    # 整理表格
    df4 = df3.drop(columns=['獲利年成長率','各項資產佔總資產比重','資產季成長率','資產年成長率','負債&股東權益佔總資產','負債&股東權益季增減率','負債&股東權益年增減率','償債能力','經營能力','現金流量狀況','其他指標'])
    df4.columns = df4.columns.str.replace(' ', '').str.replace('  ','')
    
    # 整理並輸出
    dataExport2 = pd.concat([dataExport, df4], axis=0, ignore_index=True)
    dataExport2 = dataExport2.fillna(0)
    
    # 只选择特定列，并填充缺失值为 0
    selected_cols = ['代碼','季度','營業毛利率','營業利益率','稅前淨利率','稅後淨利率','稅後淨利率(母公司)',
'每股稅前盈餘(元)','每股稅後盈餘(元)','每股淨值(元)','股東權益報酬率(季累計)','股東權益報酬率(年預估)','資產報酬率(季累計)',
'資產報酬率(年預估)','營收年成長率','毛利年成長率','營業利益年成長率','稅前淨利年成長率','稅後淨利年成長率','稅後淨利年成長率(母公司)',
'每股稅後盈餘年成長率','現金(%)','應收帳款(%)','存貨(%)','速動資產(%)','流動資產(%)','基金與投資(%)','固定資產(%)',
'無形資產(%)','其他資產(%)','現金季成長率','應收帳款季成長率','存貨季成長率','流動資產季成長率','基金與投資季成長率',
'固定資產季成長率','無形資產季成長率','其他資產季成長率','資產總額季成長率','現金年成長率','應收帳款年成長率',
'存貨年成長率','流動資產年成長率','基金與投資年成長率','固定資產年成長率','無形資產年成長率','其他資產年成長率',
'資產總額年成長率','應付帳款(%)','流動負債(%)','長期負債(%)','其他負債(%)','負債總額(%)','普通股股本(%)','股東權益總額(%)',
'應付帳款季成長率','流動負債季成長率','長期負債季成長率','其他負債季成長率','負債總額季成長率','普通股股本季成長率',
'股東權益總額季成長率','應付帳款年成長率','流動負債年成長率','長期負債年成長率','其他負債年成長率','負債總額年成長率',
'普通股股本年成長率','股東權益總額年成長率','現金比','速動比','流動比','利息保障倍數','現金流量比(季累計)',
'現金流量比(年預估)','營業成本率','營業費用率','應收帳款週轉率(次/年)','應收款項收現日數(日)','應付帳款週轉率(次/年)',
'應付款項付現日數(日)','存貨週轉率(次/年)','平均售貨日數(日)','固定資產週轉率(次/年)','總資產週轉率(次/年)',
'淨值週轉率(次/年)','應收帳款佔營收比率(季累計)','應收帳款佔營收比率(年預估)','存貨佔營收比率(季累計)',
'存貨佔營收比率(年預估)','每股營業現金流量(元)','每股投資現金流量(元)','每股融資現金流量(元)','每股淨現金流量(元)',
'每股自由現金流量(元)','負債對淨值比率','長期資金適合率','所得稅佔稅前淨利比率','業外損益佔營收比率',
'業外損益佔稅前淨利比率','財報評分(100為滿分)']
    dataExport2 = dataExport2.loc[:, selected_cols].fillna(0)
    
    return dataExport2
    
from tqdm import tqdm  
import random

if __name__ == "__main__":
    連接到資料庫('goodinfoRevenue.db')
    thisMonthFinancialStatements = '2022Q4'

    errorID = []

    for id in tqdm(ids):
        # 先查詢本地資料庫
        dfdatabase = pd.read_sql(sql, conn)
        dfdatabaseMask = dfdatabase['代碼'] == id
        dfdatabase2 = dfdatabase[dfdatabaseMask]
        
        if thisMonthFinancialStatements not in dfdatabase2['季度'].values:
            dataExport3 = 爬蟲(id)
            time.sleep(int(format(random.randint(10,20))))
            
            for index, row in dataExport3.iterrows():
                try:
                    cursor.execute('INSERT INTO financialStatements1 values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', row)
                    conn.commit()
                except:
                    errorID.append(id)
                    print(f'錯誤代碼{id}')
                    pass
                # cursor.execute('INSERT INTO financialStatements1 values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', row)
                # conn.commit()
        else:
            pass
        
        
        
    print('爬蟲完畢')
    conn.close()

  0%|          | 3/1753 [00:29<4:48:12,  9.88s/it]


KeyError: "['長期負債(%)', '長期負債季成長率', '長期負債年成長率', '長期資金適合率'] not in index"

In [9]:
id

'1104'

In [7]:
dataExport3 = 爬蟲('1102')
dataExport3


,代碼,季度,營業毛利率,營業利益率,稅前淨利率,稅後淨利率,稅後淨利率(母公司),每股稅前盈餘(元),每股稅後盈餘(元),每股淨值(元),...,每股自由現金流量(元),負債對淨值比率,長期資金適合率,所得稅佔稅前淨利比率,業外損益佔營收比率,業外損益佔稅前淨利比率,財報評分(100為滿分),營業準備(%),營業準備季成長率,營業準備年成長率
0,1102,2022Q4,13.01,9.49,16.82,13.94,13.3,4.59,3.63,45.35,...,4.94,76,602.5,17.11,7.33,43.6,45,0.32,0.91,1.27
1,1102,2022Q3,15,11.34,21.42,18.09,17.26,4.27,3.44,45.55,...,4.17,76.7,592.5,15.54,10.08,47.06,47,0.32,0.66,7.05
2,1102,2022Q2,16.14,12.44,20.5,16.85,15.83,2.73,2.11,43.63,...,3.97,83.29,564.7,17.77,8.06,39.3,47,0.31,0.37,9.35
3,1102,2022Q1,15.12,12.52,17.45,13.8,12.87,1.03,0.76,46.51,...,1.66,69.93,575.7,20.88,4.93,28.24,47,0.32,-0.67,11.69
4,1102,2021Q4,23.45,19.58,24.28,19.4,16.68,6.84,4.7,44.26,...,2.16,75.88,561.1,20.09,4.7,19.37,53,0.32,6.67,12.23
5,1102,2021Q3,24.07,20.06,26.22,20.8,17.72,5.18,3.5,42.8,...,2.32,76.36,569.6,20.66,6.16,23.49,57,0.31,2.83,33.84
6,1102,2021Q2,26.23,22.41,28.66,22.87,19.14,3.68,2.46,42.34,...,1.49,86.39,419.8,20.2,6.25,21.8,60,0.31,2.51,27.92
7,1102,2021Q1,24.39,20.83,26.54,21.53,17.9,1.51,1.02,44.87,...,0.42,71.35,439.4,18.86,5.71,21.52,55,0.31,-0.18,25.96
8,1102,2020Q4,29.82,25.14,30.86,23.99,18.8,7.71,4.7,43.96,...,4.83,74.87,415.8,22.24,5.72,18.53,57,0.31,27.21,28
9,1102,2020Q3,28.92,23.97,30.9,23.8,18.95,5.38,3.3,42.38,...,3.89,81.21,422.7,22.98,6.94,22.45,60,0.24,-1.72,4.1


In [ ]:
dataExport = pd.DataFrame()
dataExport = pd.DataFrame(columns=['代碼','季度','營業毛利率','營業利益率','稅前淨利率','稅後淨利率','稅後淨利率(母公司)',
'每股稅前盈餘(元)','每股稅後盈餘(元)','每股淨值(元)','股東權益報酬率(季累計)','股東權益報酬率(年預估)','資產報酬率(季累計)',
'資產報酬率(年預估)','營收年成長率','毛利年成長率','營業利益年成長率','稅前淨利年成長率','稅後淨利年成長率','稅後淨利年成長率(母公司)',
'每股稅後盈餘年成長率','現金(%)','應收帳款(%)','存貨(%)','速動資產(%)','流動資產(%)','基金與投資(%)','固定資產(%)',
'無形資產(%)','其他資產(%)','現金季成長率','應收帳款季成長率','存貨季成長率','流動資產季成長率','基金與投資季成長率',
'固定資產季成長率','無形資產季成長率','其他資產季成長率','資產總額季成長率','現金年成長率','應收帳款年成長率',
'存貨年成長率','流動資產年成長率','基金與投資年成長率','固定資產年成長率','無形資產年成長率','其他資產年成長率',
'資產總額年成長率','應付帳款(%)','流動負債(%)','長期負債(%)','其他負債(%)','負債總額(%)','普通股股本(%)','股東權益總額(%)',
'應付帳款季成長率','流動負債季成長率','長期負債季成長率','其他負債季成長率','負債總額季成長率','普通股股本季成長率',
'股東權益總額季成長率','應付帳款年成長率','流動負債年成長率','長期負債年成長率','其他負債年成長率','負債總額年成長率',
'普通股股本年成長率','股東權益總額年成長率','現金比','速動比','流動比','利息保障倍數','現金流量比(季累計)',
'現金流量比(年預估)','營業成本率','營業費用率','應收帳款週轉率(次/年)','應收款項收現日數(日)','應付帳款週轉率(次/年)',
'應付款項付現日數(日)','存貨週轉率(次/年)','平均售貨日數(日)','固定資產週轉率(次/年)','總資產週轉率(次/年)',
'淨值週轉率(次/年)','應收帳款佔營收比率(季累計)','應收帳款佔營收比率(年預估)','存貨佔營收比率(季累計)',
'存貨佔營收比率(年預估)','每股營業現金流量(元)','每股投資現金流量(元)','每股融資現金流量(元)','每股淨現金流量(元)',
'每股自由現金流量(元)','負債對淨值比率','長期資金適合率','所得稅佔稅前淨利比率','業外損益佔營收比率',
'業外損益佔稅前淨利比率','財報評分(100為滿分)'])